In [13]:
import logging
from typing import Literal, List, Dict, Annotated
from langchain_core.tools import tool
import re
import httpx
import os
from dotenv import load_dotenv
from typing import Optional
from pydantic import BaseModel, Field
from utils import constants as CONST


# Setup the logging configuration
log_level = getattr(logging,CONST.LOG_LEVEL )
logging.basicConfig(
    level=log_level,
    format="%(asctime)s - %(levelname)s - %(filename)s - Line: %(lineno)d - %(message)s"
)
logger = logging.getLogger()

try:
    # STEP-1: Read the env variables
    WD_BEARER_TOKEN = os.environ["WD_BEARER_TOKEN"]
    WD_VERSION = os.environ["WD_VERSION"]
    WD_URL = os.environ["WD_URL"]
    WD_RAN_PROJECT_ID = os.environ["WD_RAN_PROJECT_ID"]
    URL_FOR_PDF_LINKS = os.environ["URL_FOR_PDF_LINKS"]
    DOC_COUNT = int(os.environ["DOC_COUNT"])
    WD_PASSAGE_CHARACTERS_LIMIT = int(os.environ["WD_PASSAGE_CHARACTERS_LIMIT"])
except Exception as e:
    logger.error(e)
    logger.info("Loading Environmment Variables from local .env file")
    load_dotenv()
    WD_BEARER_TOKEN = os.environ["WD_BEARER_TOKEN"]
    WD_VERSION = os.environ["WD_VERSION"]
    WD_URL = os.environ["WD_URL"]
    WD_RAN_PROJECT_ID = os.environ["WD_RAN_PROJECT_ID"]
    URL_FOR_PDF_LINKS = os.environ["URL_FOR_PDF_LINKS"]
    DOC_COUNT = int(os.environ["DOC_COUNT"])
    WD_PASSAGE_CHARACTERS_LIMIT = int(os.environ["WD_PASSAGE_CHARACTERS_LIMIT"])
# Tool decorator applied externally

async def vector_search(query: str, vendor: str = None, release: str = None, category: str = None) -> str:
    """
    Sync Wrapper function for  tool that performs vector search

    Args:
        query (str): Search query text

    Returns:
        str: Formatted search results
    """
    payload = {
        "aggregation": "[term(enriched_text.entities.text,name:entities)]",
        "count": DOC_COUNT,
        "return": ["title", "metadata", "extracted_metadata"],
        "passages": {
            "fields": ["text"],
            "enabled": True,
            "characters": WD_PASSAGE_CHARACTERS_LIMIT,
            "per_document": True,
            "find_answers": False,
        },
        "natural_language_query": query,
        "table_results": {
            "enabled": False
        },
        "sort" : "-metadata.file_last_modified"
    }

        # Initialize an empty list to store filter conditions
    filter_conditions = []

    # Add conditions if they exist
    if vendor:
        filter_conditions.append(f'metadata.vendor:"{vendor}"')
    if release:
        filter_conditions.append(f'metadata.release:"{release}"')
    if category:
        filter_conditions.append(f'metadata.file_category:"{category}"')

    # Join all conditions with comma (AND operator) and add to payload if any exist
    if filter_conditions:
        payload["filter"] = ','.join(filter_conditions)
        logger.info(f'filter ::::{payload["filter"]}')

    # Add sorting ONLY if `release` was not passed
    # if not release:
    #     payload["sort"] = "-metadata.file_last_modified"
        # # Sort by release descending, then timestamp descending
        # payload["sort"] = "-metadata.release,-metadata.timestamp"

    # URL and endpoint
    url = f'{WD_URL}/v2/projects/{WD_RAN_PROJECT_ID}/query?version={WD_VERSION}'

    # Headers
    headers = {
        "Accept": "application/json",
        "Content-Type": "application/json",
        "Authorization": f'Bearer {WD_BEARER_TOKEN}'
    }

    # Make the HTTP POST request (synchronously)
    async with httpx.AsyncClient(verify=False, timeout=60) as client:
        response = await client.post(url, headers=headers, json=payload)

        # Check response
        #print(f"Status Code: {response.status_code}")
        try:
            res = response.json()
            #print(f"Response: {res}")

            results = res.get('results', [])
            print("<-----------------results------------------>")
            print(results)
            print("<-----------------results------------------>")
            formatted_results = []
            for result in results:
                formatted_results.append(f"file_name :: {result.get('metadata')['file_name']}")
                #formatted_results.append("\n")
                #formatted_results.append(f"s3_source_URL :: {URL_FOR_PDF_LINKS}?s3_file_url={result.get('metadata')['s3_url']}")
                formatted_results.append("----->")
                cleaned_text = re.sub(r'[-\s]+', ' ', result.get('document_passages')[0]['passage_text']) \
                    .replace("<em>", "").replace("</em>", "").strip().replace("\n", "").replace("\r", "")
                formatted_results.append(cleaned_text)
                formatted_results.append("\n")
                #formatted_results.append("\n")

            # Return formatted results
            return "\n".join(formatted_results)

        except Exception as e:
            logger.error(f"Error parsing response: {e}")
            logger.error(f"Response text: {response.text}")
            return "Error occurred while processing the search."

query = "Cells Disabled on Samsung DU"
results = await vector_search(query, 'mavenir')

2025-05-05 18:35:15,752 - INFO - 1642811248.py - Line: 85 - filter ::::metadata.vendor:"mavenir"
2025-05-05 18:35:17,053 - INFO - _client.py - Line: 1786 - HTTP Request: POST https://cpd-cpd-inst-operands.apps.watsonx.cpni-wl-dw-d.aws.dishcloud.io/discovery/cpd-inst-operands-wd/instances/1730206863041206/api/v2/projects/826cc215-0482-454f-b649-217416edb7e3/query?version=2022-08-01 "HTTP/1.1 200 OK"


<-----------------results------------------>
[{'document_id': 'ea3a34dd-8663-4947-97f3-934e1ef046bd', 'result_metadata': {'collection_id': '8a59273c-3e71-5716-0000-019449f92652', 'document_retrieval_source': 'search'}, 'metadata': {'file_name': 'Mavenir Troubleshooting Guide.docx', 'release': 'NOT_DEFINED', 's3_url': 's3://dl-dish-wrls-whlsl-network-documents-cpni-p/troubleshooting_guides/mavenir_troubleshooting_guides/Mavenir Troubleshooting Guide.docx', 'parent_document_id': 'ea3a34dd-8663-4947-97f3-934e1ef046bd', 'file_last_modified': '2024-11-07 17:11:21+00:00', 'xls_tab': 'NOT_DEFINED', 'version': 'NOT_DEFINED', 'file_category': 'other', 'bucket': 'dl-dish-wrls-whlsl-network-documents-cpni-p', 'file_key': 'troubleshooting_guides/mavenir_troubleshooting_guides/Mavenir Troubleshooting Guide.docx', 'node': 'NOT_DEFINED', 'parent_folder': 'troubleshooting_guides', 'size': 4870942, 'file_type': 'docx', 'vendor': 'mavenir'}, 'extracted_metadata': {'sha1': 'A86BED99D9EC0605347C81C972FA01

In [6]:
results

'file_name :: Mavenir Troubleshooting Guide.docx\n----->\nme mtcil1 mvnr scO2\\me mtcil1 ● Check CSR Interface State:(change this to 7.7) Page 13 of 26 Interface Status Protocol Description BV95 BV96 BV201 BV311 BV318 BV323 LoO Lo2 Lo10 NuO §§§§§§§§§ BMC mgmt ESX1 host mgmt Maveni r M Plane Mavenir K8s mgmt Maveni r Fl C Mavenir F1 U Routing SDN C SERVICE MGMT vrf Loopback 28/878/SPu0/0 G1 G1 G1 ooo0s admin dowr up up admin down up aown aown huO huQ Te0/0 TeO Te0, TeO TeO TeO 3 up admin down admin down admin down up admi n down admi n down admin down SiteBoss Rectifi er controller server BMC TeO Te0, 4 200 4 201 4 202 200 201 202 200 6. 201 TeO TeO TeO Te0/0 TeO Te0, TeO TeO ~o 202 TeO TeO Te0, TeO Te0 TeO 7 200 7 201 7 202 8 gooccaaa 200 201 202 200 201 202 10 10 100 200 0/11 peeeseeseeeseeseessessesssssgtte RU1 RU1 S P1 ane RU1 Mavenir M Plane RU1 Mavenir CU Plane RU2 RU2 S P1 ane RU2 Mavenir M Plane RU2 Mavenir CU Plane RU3 RU3 S P1 ane RU3 Mavenir M Plane RU3 Mavenir CU Plane RU4 R

In [1]:
%pwd

'/Users/prakashb/Desktop/Dish/dish_rca_space/dish-ran'

In [2]:
import os
from langchain_core.tools import tool
from typing import Annotated, Dict, List
from dotenv import load_dotenv
import traceback
from prompts.ran2_qa.ran_sql_prompts import (
                                    mcms_cm_ret_state_12hr_prompt,
                                    mcms_cm_topology_state_du_12hr_prompt,
                                    mcms_cm_topology_state_cuup_12hr_prompt,
                                    mcms_cm_topology_state_cucp_12hr_prompt,
                                    mcms_cm_topology_state_rru_12hr_prompt,
                                    table_name_gpl_1_prompt_mistral, table_name_gpl_2_prompt_mistral, table_name_gpl_3_prompt_mistral,
                                    table_name_gpl_4_prompt_mistral, table_name_gpl_5_prompt_mistral, table_name_gpl_6_prompt_mistral,
                                    usm_cm_config_cucp_1d_prompt, usm_cm_config_du_1d_prompt, usm_cm_ret_state_1d_prompt, 
                                    table_name_gpl_1_prompt_mistral_misaligned_params, table_identify_misaligned_params
                                    )
from utils.logger import logger
# from utils.ran2_qa.ran_sql_athena_utils import query_athena_db
from utils.ran2_qa.ran_sql_postgres_utils import query_postgres_db
from utils.ran2_qa.ran_sql_postgres_utils import query_postgres_db as query_athena_db # used for testing locally
from llms.ran2_qa.ran_part_two_llms import llm_ran_text_to_sql, llm_ran_table_identify

try:
    WATSONX_MODEL_ID_TEXT_TO_SQL_MISTRAL = os.environ['WATSONX_MODEL_ID_TEXT_TO_SQL_MISTRAL']
    WATSONX_MODEL_ID_TEXT_TO_SQL_MISTRAL_PARAMS = os.environ['WATSONX_MODEL_ID_TEXT_TO_SQL_MISTRAL_PARAMS']
    WATSONX_MODEL_ID_TEXT_TO_SQL_LLAMA = os.environ['WATSONX_MODEL_ID_TEXT_TO_SQL_LLAMA']
    WATSONX_MODEL_ID_TEXT_TO_SQL_LLAMA_PARAMS = os.environ['WATSONX_MODEL_ID_TEXT_TO_SQL_LLAMA_PARAMS']
    WATSONX_PROJECT_ID = os.environ['WATSONX_PROJECT_ID']
    WATSONX_URL= os.environ['WATSONX_URL']
    WATSONX_API_KEY=os.environ['WATSONX_API_KEY']
    mcms_cm_topology_state_du_12hr_sub_id=os.environ['mcms_cm_topology_state_du_12hr_sub_id']
    mcms_cm_ret_state_12hr_sub_id=os.environ['mcms_cm_ret_state_12hr']
    mcms_cm_topology_state_cuup_12hr_sub_id=os.environ['mcms_cm_topology_state_cuup_12hr']
    mcms_cm_topology_state_cucp_12hr_sub_id=os.environ['mcms_cm_topology_state_cucp_12hr']
    mcms_cm_topology_state_rru_12hr_sub_id=os.environ['mcms_cm_topology_state_rru_12hr']
    usm_cm_config_cucp_1d_sub_id=os.environ['usm_cm_config_cucp_1d']
    usm_cm_config_du_1d_sub_id=os.environ['usm_cm_config_du_1d']
    table_name_gpl_1_sub_id=os.environ['table_name_gpl_1']
    table_name_gpl_2_sub_id=os.environ['table_name_gpl_2']
    table_name_gpl_3_sub_id=os.environ['table_name_gpl_3']
    table_name_gpl_4_sub_id=os.environ['table_name_gpl_4']
    table_name_gpl_5_sub_id=os.environ['table_name_gpl_5']
    table_name_gpl_6_sub_id=os.environ['table_name_gpl_6']
    usm_cm_ret_state_1d_sub_id=os.environ['usm_cm_ret_state_1d']

except Exception as e:
    print(e)
    load_dotenv()
    print("Loading Environmment Variables from local .env file")
    WATSONX_MODEL_ID_TEXT_TO_SQL_MISTRAL = os.environ['WATSONX_MODEL_ID_TEXT_TO_SQL_MISTRAL']
    WATSONX_MODEL_ID_TEXT_TO_SQL_MISTRAL_PARAMS = os.environ['WATSONX_MODEL_ID_TEXT_TO_SQL_MISTRAL_PARAMS']
    WATSONX_MODEL_ID_TEXT_TO_SQL_LLAMA = os.environ['WATSONX_MODEL_ID_TEXT_TO_SQL_LLAMA']
    WATSONX_MODEL_ID_TEXT_TO_SQL_LLAMA_PARAMS = os.environ['WATSONX_MODEL_ID_TEXT_TO_SQL_LLAMA_PARAMS']
    WATSONX_PROJECT_ID = os.environ['WATSONX_PROJECT_ID']
    WATSONX_URL= os.environ['WATSONX_URL']
    WATSONX_API_KEY=os.environ['WATSONX_API_KEY']
    mcms_cm_topology_state_du_12hr_sub_id=os.environ['mcms_cm_topology_state_du_12hr_sub_id']
    mcms_cm_ret_state_12hr_sub_id=os.environ['mcms_cm_ret_state_12hr']
    mcms_cm_topology_state_cuup_12hr_sub_id=os.environ['mcms_cm_topology_state_cuup_12hr']
    mcms_cm_topology_state_cucp_12hr_sub_id=os.environ['mcms_cm_topology_state_cucp_12hr']
    mcms_cm_topology_state_rru_12hr_sub_id=os.environ['mcms_cm_topology_state_rru_12hr']
    usm_cm_config_cucp_1d_sub_id=os.environ['usm_cm_config_cucp_1d']
    usm_cm_config_du_1d_sub_id=os.environ['usm_cm_config_du_1d']
    table_name_gpl_1_sub_id=os.environ['table_name_gpl_1']
    table_name_gpl_2_sub_id=os.environ['table_name_gpl_2']
    table_name_gpl_3_sub_id=os.environ['table_name_gpl_3']
    table_name_gpl_4_sub_id=os.environ['table_name_gpl_4']
    table_name_gpl_5_sub_id=os.environ['table_name_gpl_5']
    table_name_gpl_6_sub_id=os.environ['table_name_gpl_6']
    usm_cm_ret_state_1d_sub_id=os.environ['usm_cm_ret_state_1d']

In [3]:
def util_misalignment(user_question):
    #0. Generate sql query for user question
    gpl_sql_query = llm_ran_text_to_sql(user_question, table_name_gpl_1_prompt_mistral_misaligned_params, table_name_gpl_1_sub_id)
    #1. Fetch sql query for dish_gpl
    recom_params = query_postgres_db(gpl_sql_query)
    #2. Table selection from athena
    results = llm_ran_table_identify(user_question, table_identify_misaligned_params)
    #3. Columns selection
    print("####results---->")
    print(results)
    print(type(results))

    
    #4. Fetch the data and compare
    #5. Filter out and identify only the misaligned
    #6. Response creation
    return results

In [4]:
user_question = "What parameters misalignments do we have in CGI?"
sql_result = util_misalignment(user_question)

/Users/prakashb/Desktop/Dish/dish_rca_space/dish-ran/llms/ran2_qa/ran_part_two_llms.py:64: RuntimeWarning: coroutine 'store_summarization_payload_record' was never awaited
  payload_logging.store_summarization_payload_record(
2025-04-21 13:10:12 - INFO - user_question:What parameters misalignments do we have in CGI? :: query:SELECT DISTINCT ON (param_name) param_name, dish_gpl_value as recommended, version FROM ran.dish_gpl_params WHERE hierarchy = 'CGI' ORDER BY param_name, dish_gpl_value, version DESC; - llm_ran_text_to_sql
2025-04-21 13:10:14 - INFO - query:SELECT DISTINCT ON (param_name) param_name, dish_gpl_value as recommended, version FROM ran.dish_gpl_params WHERE hierarchy = 'CGI' ORDER BY param_name, dish_gpl_value, version DESC; :: sql_results:[] :: - query_postgres_db
2025-04-21 13:10:15 - INFO - user_question:What parameters misalignments do we have in CGI? :: query:Table: "dl_silver_ran_mavenir_piiprod"."mcms_cm_du_cells_admin_state_1d"
Columns: ['ran'] - llm_ran_table_id

####results---->
Table: "dl_silver_ran_mavenir_piiprod"."mcms_cm_du_cells_admin_state_1d"
Columns: ['ran']
<class 'str'>


In [5]:
sql_result

'Table: "dl_silver_ran_mavenir_piiprod"."mcms_cm_du_cells_admin_state_1d"\nColumns: [\'ran\']'

In [12]:
import ast

# Exact LLM response
response = 'Table: "dl_silver_ran_mavenir_piiprod"."mcms_cm_du_cells_admin_state_1d"\nColumns: [\'NA\']'

# Split lines
lines = response.splitlines()

# Extract table
table_line = next(line for line in lines if line.startswith("Table:"))
table = table_line.replace("Table:", "").strip().replace('"', '')

# Extract columns
columns_line = next(line for line in lines if line.startswith("Columns:"))
columns = ast.literal_eval(columns_line.replace("Columns:", "").strip())

column_check = False
if len(columns) > 1:
    column_check = False
elif len(columns) == 1:
    if columns[0] == "NA":
        column_check = True

    
lines, table, columns, column_check

(['Table: "dl_silver_ran_mavenir_piiprod"."mcms_cm_du_cells_admin_state_1d"',
  "Columns: ['NA']"],
 'dl_silver_ran_mavenir_piiprod.mcms_cm_du_cells_admin_state_1d',
 ['NA'],
 True)

In [ ]:
import ast

# Exact LLM response
response = 'Table: "dl_silver_ran_mavenir_piiprod"."mcms_cm_du_cells_admin_state_1d"\nColumns: [\'ran\']'

# Split lines
lines = response.splitlines()

# Extract table
table_line = next(line for line in lines if line.startswith("Table:"))
table = table_line.replace("Table:", "").strip().replace('"', '')

# Extract columns
columns_line = next(line for line in lines if line.startswith("Columns:"))
columns = ast.literal_eval(columns_line.replace("Columns:", "").strip())

if len(columns) > 1:
    column_check = False
elif len(columns) == 1:
    if columns[0] == "NA":
        column_check = True

# Simulated column data — replace this with your actual values
column_data = {
    'ran': ['NA']  # change values as needed
}

# Check if column has only 'NA'
column_check = "NA" if all(val == 'NA' for val in column_data[columns[0]]) else None

# Final output
print("table:", table)
print("columns:", columns)
print("column_check:", column_check)

table: dl_silver_ran_mavenir_piiprod.mcms_cm_du_cells_admin_state_1d
columns: ['ran']
column_check: NA
